In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df_1 = pd.read_csv('20250624_timeslot_1_cleaned.csv').drop('gps_X', axis=1).drop('gps_Y', axis=1).drop('Time', axis=1).drop('filename', axis=1)
df_1_d =  pd.read_csv('20250624_timeslot_1_derivative_data_cleaned.csv')

df_2 = pd.read_csv('20250624_timeslot_2_cleaned.csv').drop('gps_X', axis=1).drop('gps_Y', axis=1).drop('Time', axis=1).drop('filename', axis=1)
df_2_d =  pd.read_csv('20250624_timeslot_2_derivative_data_cleaned.csv')

df_all = pd.read_csv('20250624_all_timeslot_cleaned.csv').drop('gps_X', axis=1).drop('gps_Y', axis=1).drop('Time', axis=1).drop('filename', axis=1)
df_all_d =  pd.read_csv('20250624_all_timeslot_derivative_data_cleaned.csv')


KeyboardInterrupt



In [ ]:
df_1

In [ ]:
df_2

In [ ]:
df_all

In [ ]:
df_all_d

In [ ]:
df_1_d

In [ ]:
df_2_d

In [ ]:
## X is the actual data

In [ ]:
X_1 = df_1.iloc[:, 7:]
X_2 = df_2.iloc[:, 7:]
X_all = df_all.iloc[:, 7:]

In [ ]:
scaler_X_1 = StandardScaler()
X_1_scaled = scaler_X_1.fit_transform(X_1)
X_1_scaled = pd.DataFrame(X_1_scaled, columns=X_1.columns)

scaler_X_2 = StandardScaler()
X_2_scaled = scaler_X_2.fit_transform(X_2)
X_2_scaled = pd.DataFrame(X_2_scaled, columns=X_2.columns)

scaler_X_all = StandardScaler()
X_all_scaled = scaler_X_all.fit_transform(X_all)
X_all_scaled = pd.DataFrame(X_all_scaled, columns=X_all.columns)

In [ ]:
Y_1 = df_1.iloc[:, :7]
Y_2 = df_2.iloc[:, :7]
Y_all = df_all.iloc[:, :7]

In [ ]:
scaler_Y_1 = StandardScaler()
Y_1_scaled = scaler_Y_1.fit_transform(Y_1)
Y_1_scaled = pd.DataFrame(Y_1_scaled, columns=Y_1.columns)

scaler_Y_2 = StandardScaler()
Y_2_scaled = scaler_Y_2.fit_transform(Y_2)
Y_2_scaled = pd.DataFrame(Y_2_scaled, columns=Y_2.columns)

scaler_Y_all = StandardScaler()
Y_all_scaled = scaler_Y_all.fit_transform(Y_all)
Y_all_scaled = pd.DataFrame(Y_all_scaled, columns=Y_all.columns)


In [ ]:
## X_d is the 

In [ ]:
X_1_d = df_1_d.iloc[:, 7:]
X_2_d = df_2_d.iloc[:, 7:]
X_all_d = df_all_d.iloc[:, 7:]

In [ ]:
scaler_X_1_d = StandardScaler()
X_1_d_scaled = scaler_X_1_d.fit_transform(X_1_d)
X_1_d_scaled = pd.DataFrame(X_1_d_scaled, columns=X_1_d.columns)

scaler_X_2_d = StandardScaler()
X_2_d_scaled = scaler_X_2_d.fit_transform(X_2_d)
X_2_d_scaled = pd.DataFrame(X_2_d_scaled, columns=X_2_d.columns)

scaler_X_all_d = StandardScaler()
X_all_d_scaled = scaler_X_all_d.fit_transform(X_all_d)
X_all_d_scaled = pd.DataFrame(X_all_d_scaled, columns=X_all_d.columns)

In [ ]:
Y_1_d = df_1_d.iloc[:, :7]
Y_2_d = df_2_d.iloc[:, :7]
Y_all_d = df_all_d.iloc[:, :7]

In [ ]:
scaler_Y_1_d = StandardScaler()
Y_1_d_scaled = scaler_Y_1_d.fit_transform(Y_1_d)
Y_1_d_scaled = pd.DataFrame(Y_1_d_scaled, columns=Y_1_d.columns)

scaler_Y_2_d = StandardScaler()
Y_2_d_scaled = scaler_Y_2_d.fit_transform(Y_2_d)
Y_2_d_scaled = pd.DataFrame(Y_2_d_scaled, columns=Y_2_d.columns)

scaler_Y_all_d = StandardScaler()
Y_all_d_scaled = scaler_Y_all_d.fit_transform(Y_all_d)
Y_all_d_scaled = pd.DataFrame(Y_all_d_scaled, columns=Y_all_d.columns)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # # Set the size of the figure
# plt.figure(figsize=(15, 10))

# # Create the heatmap with enhancements
# sns.heatmap(X_2.corr(), annot=True, fmt=".2f", cmap='coolwarm', 
#             linewidths=0.5,)

# # Add a title with enhanced formatting
# plt.title('Correlation Heatmap', fontweight='bold')

# # Show the plot
# plt.show()

## 用NN测试关系

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    explained_variance_score, median_absolute_error
)
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Improved Neural Network with dropout and GPU support
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 128)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

def evaluate_NN(X, y, test_size=0.05, n_epochs=300, lr=1e-3, name='VOC', random_state=42):
    # Set random seed for reproducibility
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Split into train/test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, shuffle=True)
    print(f'Training size: {len(X_train)}, Testing size: {len(X_test)}')

    # Convert to torch tensors (no scaling)
    X_train_torch = torch.tensor(X_train.values, dtype=torch.float32, device=device)
    y_train_torch = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32, device=device)
    X_test_torch = torch.tensor(X_test.values, dtype=torch.float32, device=device)

    # Model, loss, optimizer
    model = NeuralNetwork(input_size=X.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    model.train()
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        outputs = model(X_train_torch)
        loss = criterion(outputs, y_train_torch)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 50 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_torch).cpu().numpy().flatten()

    # Calculate metrics
    r2_model = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    explained_var = explained_variance_score(y_test, y_pred)
    median_ae = median_absolute_error(y_test, y_pred)

    # Baseline model: Mean prediction
    y_baseline = np.mean(y_train)
    y_baseline_pred = np.full(y_test.shape, y_baseline)
    r2_baseline = r2_score(y_test, y_baseline_pred)

    # Adjusted R²
    n = X_test.shape[0]
    p = X_test.shape[1]
    adjusted_r2 = 1 - (1 - r2_model) * (n - 1) / (n - p - 1)

    # Print results
    print(f"R² for Baseline Model: {r2_baseline:.4f}")
    print(f"R² for Neural Network: {r2_model:.4f}")
    print(f"Adjusted R² for Neural Network: {adjusted_r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Explained Variance Score: {explained_var:.4f}")
    print(f"Median Absolute Error: {median_ae:.4f}")

    # Visualization
    sns.set(style="whitegrid")
    fig, axs = plt.subplots(2, 2, figsize=(18, 14))

    # 1. Scatter plot: True vs Predicted
    axs[0, 0].scatter(y_test, y_pred, alpha=0.6, color="#0072B2", edgecolor='k', s=60)
    axs[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
    axs[0, 0].set_xlabel("True Values", fontsize=14)
    axs[0, 0].set_ylabel("Predicted Values", fontsize=14)
    axs[0, 0].set_title(f"True vs Predicted Values ({name})", fontsize=16, fontweight='bold')
    axs[0, 0].grid(True, which='both', linestyle='--', linewidth=0.5)
    axs[0, 0].tick_params(axis='both', labelsize=12)
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    axs[0, 0].set_xlim(min_val, max_val)
    axs[0, 0].set_ylim(min_val, max_val)
    axs[0, 0].set_aspect('equal', adjustable='box')

    # 2. Prediction error plot (regression line)
    sns.regplot(
        x=y_test, y=y_pred, ax=axs[0, 1],
        scatter_kws={"alpha": 0.6, "color": "#D55E00", "edgecolor": "k", "s": 60},
        line_kws={"color": "navy", "lw": 3}
    )
    axs[0, 1].set_xlabel("True Values", fontsize=14)
    axs[0, 1].set_ylabel("Predicted Values", fontsize=14)
    axs[0, 1].set_title(f"Prediction Error Plot ({name})", fontsize=16, fontweight='bold')
    axs[0, 1].grid(True, linestyle='--', linewidth=0.5)
    axs[0, 1].tick_params(axis='both', labelsize=12)

    # 3. "Feature Importances": absolute value of input layer weights
    fc1_weights = model.fc1.weight.detach().cpu().numpy()  # shape: (128, in_features)
    importances = np.mean(np.abs(fc1_weights), axis=0)  # aggregate across first hidden layer
    features = X.columns
    indices = np.argsort(importances)[::-1]
    
    # Use all features instead of top 10
    all_importances = importances[indices]
    all_features = features[indices]
    colors = sns.color_palette("coolwarm", len(all_importances))
    bars = axs[1, 0].barh(range(len(all_importances)), all_importances[::-1], color=colors[::-1], edgecolor='black')
    axs[1, 0].set_yticks(range(len(all_importances)))
    axs[1, 0].set_yticklabels(all_features[::-1], fontsize=12)
    axs[1, 0].invert_yaxis()
    axs[1, 0].set_xlabel("Input Layer Importance (abs mean)", fontsize=14)
    axs[1, 0].set_title(f"Neural Net Feature Importances ({name})", fontsize=16, fontweight='bold')
    axs[1, 0].grid(axis='x', linestyle='--', linewidth=0.5)

    # 4. Distribution of Actual vs Predicted (KDE plot)
    sns.kdeplot(y_test, label=f'Actual {name}', fill=True, ax=axs[1, 1], color="#1B9E77")
    sns.kdeplot(y_pred, label=f'Predicted {name}', fill=True, ax=axs[1, 1], color="#D95F02")
    axs[1, 1].set_title(f'Distribution of Actual vs Predicted ({name})', fontsize=16, fontweight='bold')
    axs[1, 1].set_xlabel(f'{name}', fontsize=14)
    axs[1, 1].legend(fontsize=12)
    axs[1, 1].tick_params(axis='both', labelsize=12)
    axs[1, 1].grid(True, linestyle='--', linewidth=0.5)

    plt.tight_layout()
    plt.show()
    
    return {
        'r2': r2_model,
        'adjusted_r2': adjusted_r2,
        'r2_baseline': r2_baseline,
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'explained_var': explained_var,
        'median_ae': median_ae,
    }

In [ ]:
from typing import List
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

def results_summary(
    air_type: str, 
    X_in: List[pd.DataFrame], 
    Y_in: List[pd.DataFrame], 
    X_in_names: List[str], 
    Y_in_names: List[str]
) -> None:
    """
    Generate a results summary matrix and heatmap for given X and Y inputs.
    
    Args:
        air_type (str): The target air quality type (e.g., 'VOC').
        X_in (list): List of feature input DataFrames.
        Y_in (list): List of target input DataFrames.
        X_in_names (list): Names corresponding to X_in.
        Y_in_names (list): Names corresponding to Y_in.
    """
    # Initialize an empty DataFrame for results
    results_matrix = pd.DataFrame(index=X_in_names, columns=Y_in_names)
    
    # Fill the matrix with evaluation results
    for i, x in enumerate(tqdm(X_in, desc="X Loop")):
        for j, y in enumerate(tqdm(Y_in, desc="Y Loop")):
            try:
                print(f"\n → X: {X_in_names[i]}, Y: {Y_in_names[j]}")
                metrics = evaluate_NN(x, y[air_type], name=air_type)
                results_matrix.iloc[i, j] = round(metrics['r2'], 4) 
            except ValueError as e:
                # Handle inconsistent sample size or other ValueErrors
                print(f"Skipping pair (X: {X_in_names[i]}, Y: {Y_in_names[j]}) due to error: {e}")
                results_matrix.iloc[i, j] = None  # Mark as empty or 'None'

    # Display the results matrix
    print("\nResults Summary Matrix:")
    print(results_matrix)

    # Plot a heatmap, skipping NaN values
    plt.figure(figsize=(4, 3))
    sns.heatmap(
        results_matrix.astype(float), 
        annot=True, 
        cmap='coolwarm', 
        xticklabels=Y_in_names, 
        yticklabels=X_in_names, 
        cbar_kws={'label': 'R² Value'},
        mask=results_matrix.isnull()  # Mask NaN values
    )
    plt.title(f'R² Values Heatmap for {air_type}')
    plt.xlabel('Y Inputs')
    plt.ylabel('X Inputs')
    plt.show()

# Define placeholders for potentially missing variables
X_3 = X_3_scaled = X_3_d = X_3_d_scaled = None
Y_3 = Y_3_scaled = Y_3_d = Y_3_d_scaled = None

def choose_timeslot(
    timeslot: str = '1', 
    air_type: str = 'VOC'
) -> None:
    """
    Choose the appropriate timeslot and generate results summary.

    Args:
        timeslot (str): Timeslot to use ('1', '2', '3', or 'all').
        air_type (str): The target air quality type (e.g., 'VOC').
    """
    # Define the timeslot mapping
    timeslot_mapping = {
        '1': {
            'X_input': [X_1, X_1_scaled, X_1_d, X_1_d_scaled],
            'Y_input': [Y_1, Y_1_scaled, Y_1_d, Y_1_d_scaled],
            'X_names': ['X_1', 'X_1_scaled', 'X_1_d', 'X_1_d_scaled'],
            'Y_names': ['Y_1', 'Y_1_scaled', 'Y_1_d', 'Y_1_d_scaled']
        },
        '2': {
            'X_input': [X_2, X_2_scaled, X_2_d, X_2_d_scaled],
            'Y_input': [Y_2, Y_2_scaled, Y_2_d, Y_2_d_scaled],
            'X_names': ['X_2', 'X_2_scaled', 'X_2_d', 'X_2_d_scaled'],
            'Y_names': ['Y_2', 'Y_2_scaled', 'Y_2_d', 'Y_2_d_scaled']
        },
        '3': {
            'X_input': [X_3, X_3_scaled, X_3_d, X_3_d_scaled],
            'Y_input': [Y_3, Y_3_scaled, Y_3_d, Y_3_d_scaled],
            'X_names': ['X_3', 'X_3_scaled', 'X_3_d', 'X_3_d_scaled'],
            'Y_names': ['Y_3', 'Y_3_scaled', 'Y_3_d', 'Y_3_d_scaled']
        },
        'all': {
            'X_input': [X_all, X_all_scaled, X_all_d, X_all_d_scaled],
            'Y_input': [Y_all, Y_all_scaled, Y_all_d, Y_all_d_scaled],
            'X_names': ['X_all', 'X_all_scaled', 'X_all_d', 'X_all_d_scaled'],
            'Y_names': ['Y_all', 'Y_all_scaled', 'Y_all_d', 'Y_all_d_scaled']
        }
    }
    
    # Ensure the timeslot is valid
    if timeslot not in timeslot_mapping:
        raise ValueError(f"Invalid timeslot '{timeslot}'. Choose from '1', '2', '3', or 'all'.")

    # Get inputs and names for the selected timeslot
    selected_timeslot = timeslot_mapping[timeslot]
    X_input = selected_timeslot['X_input']
    Y_input = selected_timeslot['Y_input']
    X_names = selected_timeslot['X_names']
    Y_names = selected_timeslot['Y_names']

    # Generate the results summary
    results_summary(air_type=air_type, X_in=X_input, Y_in=Y_input, X_in_names=X_names, Y_in_names=Y_names)

## Time slot 1

In [ ]:
choose_timeslot('1','VOC')

In [ ]:
choose_timeslot('1','CO')

In [ ]:
choose_timeslot('1','NO2')

In [ ]:
choose_timeslot('1','SO2')

In [ ]:
choose_timeslot('1','PM25')

In [ ]:
choose_timeslot('1','PM10')

In [ ]:
choose_timeslot('1','O3')

## Time slot 2

In [ ]:
choose_timeslot('2','VOC')

In [ ]:
choose_timeslot('2','CO')

In [ ]:
choose_timeslot('2','NO2')

In [ ]:
choose_timeslot('2','SO2')

In [ ]:
choose_timeslot('2','PM25')

In [ ]:
choose_timeslot('2','PM10')

In [ ]:
choose_timeslot('2','O3')

## Timeslot all

In [ ]:
choose_timeslot('all','VOC')

In [ ]:
choose_timeslot('all','CO')

In [ ]:
choose_timeslot('all','NO2')

In [ ]:
choose_timeslot('all','SO2')

In [ ]:
choose_timeslot('all','PM25')

In [ ]:
choose_timeslot('all','PM10')

In [ ]:
choose_timeslot('all','O3')